# Tool Use

## How to create and use simple tools with LLM

### 1. Create a very simple tool

In [1]:
# create a calculator tool that can add two numbers
def add_numbers(x: int, y: int) -> int:
    """A simple tool that adds two numbers"""
    return x + y

# create a simple text tool
def greet_user(name: str) -> str:
    """A simple tool that greets the user"""
    return f"Hello, {name}! Nice to meet you."

# create a tool that works with lists
def find_longest_word(word_list: list) -> str:
    """Find the longest word in a list of words"""
    if not word_list:
        return "List is empty"
    return max(word_list, key=len)

### 2. Describe our tools to LLM

In [2]:
# This is how we tell the LLM what our tools can do
TOOL_DESCRIPTIONS = [
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "x": {
                        "type": "integer",
                        "description": "First number to add"
                    },
                    "y": {
                        "type": "integer",
                        "description": "Second number to add"
                    }
                },
                "required": ["x", "y"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "greet_user",
            "description": "Greet a user by name",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the user to greet"
                    }
                },
                "required": ["name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "find_longest_word",
            "description": "Find the longest word in a list",
            "parameters": {
                "type": "object",
                "properties": {
                    "word_list": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of words to check"
                    }
                },
                "required": ["word_list"]
            }
        }
    }
]

### 3. Tool Mapping
Create a dictionary to map tool names to functions

In [3]:
TOOL_MAP = {
    "add_numbers": add_numbers,
    "greet_user": greet_user,
    "find_longest_word": find_longest_word
}

### 4. Let's setup the MaLLaM's client with OpenAI

In [4]:
import os
from openai import OpenAI
import json

mallam_api_key = os.getenv("MALLAM_API_KEY")
    
# create MaLLaM client
client = OpenAI(
    base_url="https://api.mesolitica.com",
    api_key=mallam_api_key
)

In [5]:
# create messages for the LLM
messages = [
    {
        "role": "system",
        "content": """You are a helpful assistant that can use tools.
        When a user asks something, use the appropriate tool to help them.
        For math, use add_numbers.
        For greetings, use greet_user.
        For finding longest words, use find_longest_word."""
    },
    {
        "role": "user",
        "content": "Can you add 5 and 3 together?"
    }
]
    
# ask LLM what to do
response = client.chat.completions.create(
    model="mallam-small",    # MaLLaM's model
    messages=messages,
    tools=TOOL_DESCRIPTIONS,
    tool_choice="auto"
)

tool_call = json.loads(response.choices[0].message.content)
print(tool_call)

{'name': 'add_numbers', 'arguments': {'x': 5, 'y': 3}}


In [6]:
# if LLM wants to use tools
tool_call = json.loads(response.choices[0].message.content)

if tool_call:
    # get tool names and arguments
    tool_name = tool_call["name"]
    tool_args = tool_call["arguments"]
            
    # use the tool
    if tool_name in TOOL_MAP:
        tool_result = TOOL_MAP[tool_name](**tool_args)
                
        # add tool results to conversation
        messages.append({
            "role": "assistant",
            "content": f"I used {tool_name} and got result: {str(tool_result)}"
        })
     
# get final response from LLM
final_response = client.chat.completions.create(
    model="mallam-small",
    messages=messages
)

tool_used = [tool_call["name"]] if tool_call else []

print(final_response.choices[0].message.content)
print("Tool used:", tool_used)

8
Tool used: ['add_numbers']


#### Now let's test the other functions

In [7]:
# greet_user test
messages = [
    {
        "role": "system",
        "content": """You are a helpful assistant that can use tools.
        When a user asks something, use the appropriate tool to help them.
        For math, use add_numbers.
        For greetings, use greet_user.
        For finding longest words, use find_longest_word."""
    },
    {
        "role": "user",
        "content": "Please greet Adham. Just show your greetings for the final output."
    }
]
    
response = client.chat.completions.create(
    model="mallam-small",
    messages=messages,
    tools=TOOL_DESCRIPTIONS,
    tool_choice="auto"
)

tool_call = json.loads(response.choices[0].message.content)
print(tool_call)

{'name': 'greet_user', 'arguments': {'name': 'Adham'}}


In [9]:
tool_call = json.loads(response.choices[0].message.content)

if tool_call:
    tool_name = tool_call["name"]
    tool_args = tool_call["arguments"]
            
    if tool_name in TOOL_MAP:
        tool_result = TOOL_MAP[tool_name](**tool_args)
        messages.append({
            "role": "assistant",
            "content": f"I used {tool_name} and got result: {str(tool_result)}"
        })
     
final_response = client.chat.completions.create(
    model="mallam-small",
    messages=messages
)

tool_used = [tool_call["name"]] if tool_call else []

print(final_response.choices[0].message.content)
print("Tool used:", tool_used)

I used greet_user and got result: Hello, Adham! Nice to meet you.
Tool used: ['greet_user']


In [13]:
# find_longest_word test
messages = [
    {
        "role": "system",
        "content": """You are a helpful assistant that can use tools.
        When a user asks something, use the appropriate tool to help them.
        For math, use add_numbers.
        For greetings, use greet_user.
        For finding longest words, use find_longest_word."""
    },
    {
        "role": "user",
        "content": "What's the longest word among: ayam, ikan, daging?"
    }
]
    
response = client.chat.completions.create(
    model="mallam-small",
    messages=messages,
    tools=TOOL_DESCRIPTIONS,
    tool_choice="auto"
)

tool_call = json.loads(response.choices[0].message.content)
print(tool_call)

{'name': 'find_longest_word', 'arguments': {'word_list': ['ayam', 'ikan', 'daging']}}


In [14]:
tool_call = json.loads(response.choices[0].message.content)

if tool_call:
    tool_name = tool_call["name"]
    tool_args = tool_call["arguments"]
            
    if tool_name in TOOL_MAP:
        tool_result = TOOL_MAP[tool_name](**tool_args)
        messages.append({
            "role": "assistant",
            "content": f"I used {tool_name} and got result: {str(tool_result)}"
        })
     
final_response = client.chat.completions.create(
    model="mallam-small",
    messages=messages
)

tool_used = [tool_call["name"]] if tool_call else []

print(final_response.choices[0].message.content)
print("Tool used:", tool_used)

I used find_longest_word and got the longest word as 'daging'.
Tool used: ['find_longest_word']


## Let's a build a Simple Customer Support Agent with LLM
This example shows how to use LLM with basic tools for customer support

In [15]:
# import required libraries
from openai import OpenAI
import os
import json
from datetime import datetime

### 1. First, let's create a simple customer database using Python dictionary
This is like a very simple database table

In [16]:
CUSTOMER_DATABASE = {
    "cust101": {
        "name": "Muhammad Ali",
        "email": "muhdali@example.com",
        "plan": "premium"
    },
    "cust102": {
        "name": "Ain Fieqah",
        "email": "ainfieqah@example.com",
        "plan": "basic"
    }
}

### 2. Create a simple knowledge base using a dictionary

In [17]:
KNOWLEDGE_BASE = {
    "login": {
        "title": "Login Help",
        "content": """
        Common login solutions:
        1. Check your caps lock
        2. Clear browser cache
        3. Try password reset
        """
    },
    "billing": {
        "title": "Billing information",
        "content": """
        Billing information:
        - Billing cycle: Monthly
        - Payment methods: Credit card, Online banking, Direct Transfer
        """
    }
}

### 3. Now let's define our tools

In [18]:
# get customer info
def get_customer_info(customer_id):
    """Get customer information from our database"""
    # check if customer exists in our database first
    if customer_id in CUSTOMER_DATABASE:
        return CUSTOMER_DATABASE[customer_id]
    else:
        return {"error": "Customer not found"}
    
def search_knowledge_base(query):
    """Search our knowledge base for helpful articles"""
    # simple search - just checks if keywords are in the article
    results = []
    
    # convert search query to lowercase for better matching
    query = query.lower()
    
    # check each article in our knowledge base
    for topic, article in KNOWLEDGE_BASE.items():
        if topic in query or query in article["content"].lower():
            results.append(article)
            
    return results


def send_email(to_email, subject, message):
    """Simulate sending an email"""
    # in a real application, this would connect to email services
    # but for now, we'll just return what would be sent
    return {
        "sent": True,
        "to": to_email,
        "subject": subject,
        "message": message,
        "time": datetime.now().isoformat()
    }

### 4. Describe the tools to the LLM

In [19]:
TOOL_DESCRIPTIONS = [
    {
        "type": "function",
        "function": {
            "name": "get_customer_info",
            "description": "Get information about a customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "string",
                        "description": "The customer's ID (e.g., cust101)"
                    }
                },
                "required": ["customer_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Search help articles for customer issues",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "What to search for"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email to a customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "to_email": {
                        "type": "string",
                        "description": "Customer's email address"
                    },
                    "subject": {
                        "type": "string",
                        "description": "Email subject"
                    },
                    "message": {
                        "type": "string",
                        "description": "Email content"
                    }
                },
                "required": ["to_email", "subject", "message"]
            }
        }
    }
]

In [20]:
TOOL_MAP = {
    "get_customer_info": get_customer_info,
    "search_knowledge_base": search_knowledge_base,
    "send_email": send_email
}

### 5. Now let's create the main function to handle customer support requests

In [21]:
def handle_customer_request(customer_id, customer_message):
    """Handle a customer support request using LLM and tools"""
    
    # initialize MaLLaM OpenAI client
    mallam_api_key = os.getenv("MALLAM_API_KEY")
    
    client = OpenAI(
        base_url="https://api.mesolitica.com",
        api_key=mallam_api_key
    )
    
    # create initial message for the LLM
    messages = [
        {
            "role": "system",
            "content": """You are a helpful customer support agent.
            First get the customer's information, then help solve their problem
            using the knowledge base. Be polite and professional.
            When using multiple tools, return them as a JSON array like this:
            [
                {"name": "tool1", "arguments": {...}},
                {"name": "tool2", "arguments": {...}}
            ]"""
        },
        {
            "role": "user",
            "content": f"Customer {customer_id} says: {customer_message}"
        }
    ]
    
    # ask LLM what to do
    response = client.chat.completions.create(
        model="mallam-small",
        messages=messages,
        tools=TOOL_DESCRIPTIONS,
        tool_choice="auto"
    )
    
    # parse the response - to handle both single tool call and array of tool calls
    content = response.choices[0].message.content
    tool_calls = json.loads(content)
    # if it's a single tool call (not in array), convert to array
    if isinstance(tool_calls, dict):
        tool_calls = [tool_calls]
            
    tools_used = []
    #execute each tool call
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["arguments"]
        tools_used.append(tool_name)
                
        if tool_name in TOOL_MAP:
            tool_result = TOOL_MAP[tool_name](**tool_args)
            messages.append({
                "role": "tool",
                "name": tool_name,
                "content": f"I used {tool_name} and got result: {str(tool_result)}"
            })
        
    final_response = client.chat.completions.create(
        model="mallam-small",
        messages=messages
    )

    return {
        "response": final_response.choices[0].message.content,
        "tools_used": tools_used
    }

### 6. Now let's test it

In [23]:
# test 1: Login issue
print("\nTest Case 1: Login Issue")
result = handle_customer_request(
    customer_id="cust101",
    customer_message="I can't log into my account. Please help!"
)
print("Response:", result["response"])
print("Tools used:", result["tools_used"])


Test Case 1: Login Issue
Response: Hello, Muhammad Ali! I'm here to assist you with logging into your account. It sounds like you're having trouble accessing it right now. Can you tell me what the issue is? Are you getting any error messages when you try to log in?
Tools used: ['get_customer_info']


In [25]:
# test 2: Billing question
print("\nTest Case 2: Billing Question")
result = handle_customer_request(
    customer_id="cust102",
    customer_message="I need to know my billing cycle and want to update my email address"
)
print("Response:", result["response"])
print("Tools used:", result["tools_used"])


Test Case 2: Billing Question
Response: Hello Ain Fieqah, your current email address on record is ainfieqah@example.com. You can update it by clicking here [update_email_link]. Additionally, you're on the basic plan with a monthly billing cycle. For more details on billing, feel free to refer to our knowledge base article [billing_information_link].
Tools used: ['get_customer_info', 'search_knowledge_base']
